In [ ]:
class LagrangeInterpolant:
    def __init__(self, x_list: list, y_list: list):
        if len(x_list) < 2 or len(y_list) < 2:
            raise ValueError("Provide at least two points to fit")
        if len(x_list) != len(set(x_list)):
            raise ValueError("Please ensure that all x-coordinates are distinct.")

        self.x_list = x_list
        self.y_list = y_list
        self.degree = len(x_list) - 1

    def __L(self, x: float, i: int):
        if i > len(self.x_list) - 1:
            raise ValueError("Provide valid index.")
        
        numerator = 1
        for j in range(self.degree + 1):
            if j != i:
                numerator *= (x - self.x_list[j])
        
        denominator = 1
        for j in range(self.degree + 1):
            if j != i:
                denominator *= (self.x_list[i] - self.x_list[j])
        
        return (numerator/denominator)

    def evaluate(self, x: float):
        res = 0
        for i in range(self.degree + 1):
            res += self.__L(x, i) * self.y_list[i]
        return res


In [ ]:
from lagrange_interpolation import LagrangeInterpolant
import matplotlib.pyplot as plt
import numpy as np

def get_uniform_nodes(n: int):
    # get x and y from function:
    x_list = []
    y_list = []
    for i in range(n + 1):
        x_list.append((2*i)/n - 1)
        y_list.append(1/(1 + 25 * (x_list[i]**2)))
    return x_list, y_list

def get_chebyshev_nodes(n: int):
    x_list = []
    y_list = []
    for i in range(n + 1):
        x = (-1 + 1)/2 + ((1 - (-1))/2) * np.cos((2 * i + 1)/(2 * (n + 1)) * np.pi)
        x_list.append(x)
        y_list.append(1/(1 + 25 * (x_list[i]**2)))
    return x_list, y_list

def add_interpolant_to_plot(n: int, x_list: list, y_list: list):
    lagrange_interpolant = LagrangeInterpolant(x_list, y_list)
    x_axis = np.linspace(x_list[0], x_list[-1], 400)
    interpolant_values = []
    for x in np.nditer(x_axis):
        interpolant_values.append(lagrange_interpolant.evaluate(x))
    plt.plot(x_axis, interpolant_values, label="Interpolant for n = {0}".format(n))

def add_function_to_plot():
    x_axis = np.linspace(-1, 1, 400)
    y_list = []
    for x in np.nditer(x_axis):
        y_list.append(1/(1 + 25 * (x**2)))
    plt.plot(x_axis, y_list, label="True function f")
    
def generate_plots(chebyshev: bool):
    for n in [6, 10 , 14]:
        if chebyshev:
            x_list, y_list = get_chebyshev_nodes(n)
        else:
            x_list, y_list = get_uniform_nodes(n)
        add_interpolant_to_plot(n, x_list, y_list)
    add_function_to_plot()
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("y")
    phrase = "Chebyshev" if chebyshev else "uniform"
    plt.title("Lagrange interpolant for various n, in case of {0} nodes".format(phrase))
    plt.savefig("{0}_plot_for_q1.png".format(phrase))
    plt.figure()


# actually execute the code:
generate_plots(False)
generate_plots(True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# define f:
def f(x):
    return np.sin(x)

def get_data_for_n(n, epsilon):
    h = 10**(n)
    if 1 + h > np.pi/2:
        M = f(1 + h)
    else:
        M = 1
    truncation_err = (M * h) / 2
    round_off_err = (2 * epsilon) / h
    tot_err = truncation_err + round_off_err
    return truncation_err, round_off_err, tot_err

def plot(min_n, max_n, epsilon):
    n_values = []
    truncation_errs = []
    round_off_errs = []
    tot_errs = []
    n_values = np.linspace(min_n, max_n, 400)
    for n in n_values:
        trun, round_off, tot = get_data_for_n(n, epsilon)
        truncation_errs.append(trun)
        round_off_errs.append(round_off)
        tot_errs.append(tot)
    plt.plot(n_values, truncation_errs, label="Truncation error")
    plt.plot(n_values, round_off_errs, label="Roundoff error")
    plt.plot(n_values, tot_errs, label="Total error")
    h_opt = np.log10(np.sqrt((4 * epsilon) / np.sin(1)))
    plt.axvline(h_opt, label="Optimal step size", color="red")
    plt.yscale('log')
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("y")

    plt.savefig("diff_plot.png")

plot(-15, 0, 10**-16)

In [ ]:
import numpy as np

class Integrator:
    @staticmethod
    def simpson(f, x_0, x_1):
        x_m = (x_0 + x_1) / 2
        factor_1 = (x_1 - x_0) / 6
        factor_2 = f(x_0) + 4 * f(x_m) + f(x_1)
        return factor_1 * factor_2

    @staticmethod
    def integrate_composite(f, x_0, x_1, quadrature_rule, granularity, open = False):
        if not open:
            points = np.linspace(x_0, x_1, granularity + 1)
        else:
            raise NotImplementedError()
        I = 0
        for i in range(points.shape[0] - 1):
            I += quadrature_rule(f, points[i], points[i + 1])
        return I

In [ ]:
from integration import Integrator
import numpy as np

# 3.1:
sigma = 5 # as an example. The result is independent of sigma.
def f(x):
    factor_1 = 1 / (sigma * np.sqrt(2 * np.pi))
    factor_2 = np.exp((-1/2) * ((x / sigma)**2))
    return factor_1 * factor_2

#a
print(Integrator.integrate_composite(f, -sigma, sigma, Integrator.simpson, 10**6))
#b
print(Integrator.integrate_composite(f, -2 * sigma, 2 * sigma, Integrator.simpson, 10**6))
#c
print(Integrator.integrate_composite(f, -3 * sigma, 3 * sigma, Integrator.simpson, 10**6))

# 3.2:
def g(x):
    y = 9 * (np.sin(x))**2 + 4 * (np.cos(x))**2
    return np.sqrt(y)

print(Integrator.integrate_composite(g, 0, 2 * np.pi, Integrator.simpson, 10**6))